In [ ]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from shapely.wkt import loads
from city_road_network.utils.utils import get_data_subdir
from city_road_network.algo.gravity_model import run_gravity_model, get_attr_error, get_prod_error
from city_road_network.config import default_crs

In [ ]:
city_name = "moscow"
data_dir = get_data_subdir(city_name)

In [ ]:
df_zones = pd.read_csv(os.path.join(data_dir, "zones_upd.csv"), index_col=0)
df_zones["geometry"] = df_zones["geometry"].apply(loads)
df_zones["centroid"] = df_zones["centroid"].apply(loads)

print(len(df_zones.loc[df_zones["production"] == 0]))
print(len(df_zones.loc[df_zones["poi_attraction"] == 0]))

df_zones.loc[df_zones["production"] == 0, "production"] = 1
df_zones.loc[df_zones["poi_attraction"] == 0, "poi_attraction"] = 1

In [ ]:
zones_gdf = gpd.GeoDataFrame(df_zones, crs=default_crs)
zones_gdf

In [ ]:
zones_gdf[zones_gdf["poi_attraction"] == 1]

In [ ]:
prod_array = np.array(zones_gdf["production"])
attr_array = np.array(zones_gdf["poi_attraction"])

In [ ]:
prod_array.sum()

In [ ]:
trip_mat = run_gravity_model(zones_gdf)

In [ ]:
prod_error = get_prod_error(trip_mat, prod_array)
attr_error = get_attr_error(trip_mat, attr_array)
print(prod_error, attr_error)

In [ ]:
np.save(os.path.join(data_dir, "trip_mat"), trip_mat)